# Fun with language modelling

Если вы пропустили лекцию, то посмотрите слайды к ней — они где-то есть. Также полезно почитать:

* [Unreasonable effectiveness of RNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) (Andrej Karpathy)
* [Официальный пример от PyTorch](https://github.com/pytorch/examples/tree/master/word_language_model)

Рекомендуется заранее всё прочитать, чтобы понять, что от вас хотят. При желании, можете переписать всё так, как подсказывает ваше сердце.

---

In [1]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
min_len = 40  # предложения с меньшим количеством символов не будут рассматриваться
max_len = 150 # предложения с большим количеством символов будут обрезаться

## Препроцессинг (3 балла)

Возьмите какие-нибудь сырые данные. Википедия, «Гарри Поттер», «Игра Престолов», тексты Монеточки, твиты Тинькова — что угодно.

Для простоты будем делать char-level модель. Выкиньте из текстов все ненужные символы (можете оставить только алфавит и, пунктуацию). Сопоставьте всем различным символам свой номер. Удобно это хранить просто в питоновском словаре (`char2idx`). Для генерации вам потребуется ещё и обратный словарь (`idx2char`). Вы что-то такое должны были писать на вступительной — можете просто переиспользовать код оттуда.

Заранее зарезервируйте айдишники под служебные символы: `<START>`, `<END>`, `<PAD>`, `<UNK>`.

Клёво будет написать отдельный класс, который делает токенизацию и детокенизацию.

In [3]:
class Vocab:
    def __init__(self, data):
        
        self.char2idx = {}
        self.idx2char = {}
        self.char2idx
        ptr_idx = 0
        for s in data:
            for c in s:
                if c not in self.char2idx:
                    self.char2idx[c] = ptr_idx
                    self.idx2char[ptr_idx] = c
                    ptr_idx += 1
    
    def tokenize(self, sequence):
        # выполните какой-то базовый препроцессинг
        # например, оставьте только алфавит и пунктуацию
        return [self.char2idx[char] for char in sequence if char in self.char2idx]
    
    def detokenize(self, sequence):
        return ''.join([self.idx2char[idx.item()] for idx in sequence])
    
    def __len__(self):
        return len(self.char2idx)

In [4]:
class TextDataset(Dataset):
    
    def __init__(self, data_path):
        print('here')
        self.data = []
        with open('book.txt', 'r') as f:
            data = f.readlines()[:1000]
            self.vocab = Vocab(data)
        cur = torch.LongTensor(torch.zeros([max_len, len(self.vocab)], dtype=torch.int64))
        ptr = 0
        with open('book.txt', 'r') as f:
            data = f.readlines()[:1000]
            for s in data:
                for c in s:
                    cur[ptr][self.vocab.tokenize(c)[0]] = 1
                    ptr += 1
                    if ptr == max_len:
                        self.data.append(cur)
                        ptr = 0
                        cur = torch.LongTensor(torch.zeros([max_len, len(self.vocab)], dtype=torch.int64))
        print('here')
        # обучите вокаб
        print('here')
        # разделите данные на отдельные сэмплы для обучения
        # (просто список из сырых строк)
        
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        sample = torch.LongTensor(sample) # сконвертируйте в LongTensor
        target = torch.LongTensor(torch.zeros(1, len(self.vocab), dtype=torch.int64))
        target = torch.cat([target, sample]) # нужно предсказать эту же последовательность со сдвигом 1
        return sample, target

Если у вас какой-то большой массив текста (скажем, статьи Википедии), вы можете просто нарезать из него кусочки фиксированной длины и так их подавать в модель.

Если же вы хотите приключений, то можно разбить этот текст на предложения (`nltk.sent_tokenize`), и тогда все примеры будут разной длины. По соображениям производительности, вы не хотите использовать самые длинные и самые короткие сэмплы, поэтому имеет смысл обрезать их по длине.

Разобьём на обучение и валидацию:

In [5]:
dataset = TextDataset('./book.txt')

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
print(len(train_set))

here
here
here
388


In [6]:
x, y = train_set[0]


## Модель (3 балла)

Примерно такое должно зайти:

* Эмбеддинг
* LSTM / GRU
* Линейный слой
* Softmax

In [7]:
class LM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout, tie_weights):
        super().__init__()
        
        self.encoder = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers, dropout=dropout)
        self.decoder = nn.Linear(hidden_dim, vocab_size)
        self.drop = nn.Dropout(dropout)
        
        if tie_weights:
            # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
            # https://arxiv.org/abs/1608.05859
            assert hidden_dim == embedding_dim
            self.decoder.weight = self.encoder.weight

        self.nhid = hidden_dim
        self.nlayers = num_layers

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, batch_size):
        # начальный хидден должен быть нулевой
        # (либо хоть какой-то константный для всего обучения)
        return torch.zeros(batch_size, hidden_dim)

## Обучение

In [8]:
epochs = 5
lr = 1e-3 
batch_size = 64
hidden_dim = 128

device = torch.device('cpu')

model = LM(
    vocab_size = len(dataset.vocab),
    embedding_dim = hidden_dim,
    hidden_dim = hidden_dim,
    num_layers = max_len,
    dropout = 0.1,
    tie_weights= True
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

train = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test = DataLoader(test_set, batch_size=batch_size)

In [14]:
for e in range(epochs):
    for x, y in train:
        x = x.transpose(0, 1)
        print(x.shape)
        model.train()
        
        
        model.zero_grad()
        
        # 0. Распакуйте данные на нужное устройство
        # 1. Инициилизируйте hidden
        hidden = model.init_hidden(len(x))
        # 2. Прогоните данные через модель, получите предсказания на каждом токене
        output, hidden = model(x, hidden)
        print(output)
        break
        # 3. Посчитайте лосс (maxlen независимых классификаций) и сделайте backward()
#         for i in range(max_len):
#             loss = criterion.forward(y[i])
        # 4. Клипните градиенты -- у RNN-ок с этим часто бывают проблемы
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.2)
        # 5. Залоггируйте лосс куда-нибудь
        
        optimizer.step()
    
    for x, y in test:
        model.eval()
         
        hidden = model.init_hidden(len(x))
        output, hidden = model(x, hidden)
        # 3. Посчитайте лосс (maxlen независимых классификаций) и сделайте backward()
        # 4. Клипните градиенты -- у RNN-ок с этим часто бывают проблемы
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.2)
        # 5. Залоггируйте лосс куда-нибудь
        
        # сдесь нужно сделать то же самое, только без backward

torch.Size([150, 64, 79])


RuntimeError: $ Torch: not enough memory: you tried to allocate 0GB. Buy new RAM! at /opt/conda/conda-bld/pytorch-cpu_1544217717168/work/aten/src/TH/THGeneral.cpp:201

## Спеллчекер (3 балла)

Из языковой модели можно сделать простенький спеллчекер: можно визуализировать лоссы на каждом символе (либо какой-нибудь другой показатель неуверенности).

Бонус: можете усреднить перплексии по словам и выделять их, а не отдельные символы.

In [ ]:
from IPython.core.display import display, HTML

def print_colored(sequence, intensities, delimeter=''):
    html = delimeter.join([
        f'<span style="background: rgb({255}, {255-x}, {255-x})">{c}</span>'
        for c, x in zip(sequence, intensities) 
    ])
    display(HTML(html))

print_colored('Налейте мне экспрессо'.split(), [0, 0, 100], ' ')

sequence = 'Эту домашку нужно сдать втечении двух недель'
intensities = [0]*len(sequence)
intensities[25] = 50
intensities[26] = 60
intensities[27] = 70
intensities[31] = 150
print_colored(sequence, intensities)

In [ ]:
def spellcheck(sequence):
    model.eval()
    
    # векторизуйте sequence; паддинги делать не нужно
    sequence = ...
    
    # прогоните модель и посчитайте лосс, но не усредняйте
    # с losses можно что-нибудь сделать для визуализации; например, в какую-нибудь степень возвести
    losses = # ...
    
    print_colored(sequence, np.array(spellcheck_losses))

In [ ]:
sequences = ['В этом претложениии очен много очепяток.', 
             'Здесь появилась лишнняя буква.', 
             'В этом предложении все нормально.', 
             'Чтонибудь пишеться чериз дефис.', 
             'Слова нрпдзх не сущесдвует.']

for sequence in sequences:
    spellcheck(sequence)

## Генерация предложений (3 балла)

* Поддерживайте hidden state при генерации. Не пересчитывайте ничего больше одного раза.
* Прикрутите температуру: это когда при сэмплировании все логиты (то, что перед софтмаксом) делятся на какое-то число (по умолчанию 1, тогда ничего не меняется). Температура позволяет делать trade-off между разнообразием и правдоподобием (подробнее — см. блог Карпатого).
* Ваша реализация должна уметь принимать строку seed — то, с чего должно начинаться сгенерированная строка.

In [ ]:
def sample(num_tokens, seed="", temperature=1.0):
    model.eval()
        
    hidden = model.init_hidden()
    input = # векторизауйте seed, чтобы на первом этап получить нужный hidden    
    
    continuation = ''
    
    for _ in range(num_tokens)
        output, hidden = model(input, hidden)
        
        token_probas = output.squeeze().div(temperature).exp().cpu()
        token = torch.multinomial(token_probas, 1)[0]
        
        continuation += # допишите соответствующий символ
        input = # обновите input
    
    return continuation

In [ ]:
beginnings = ['Шел медведь по лесу', 
              'Встретились англичанин, американец и русский. Англичанин говорит:',
              'Так вот, однажды качки решили делать ремонт',
              'Поручик Ржевский был',
              'Идёт Будда с учениками по дороге',
              'Мюллер: Штирлиц, где вы были в 1938 году?',
              'Засылают к нам американцы шпиона под видом студента',
              'Подъезжает электричка к Долгопе:']

for beginning in beginnings:
    print(f'{beginning}... {sample(10, beginning)})
    print()